In [1]:
from langgraph.graph import StateGraph, MessagesState, START, END

## langgraph

In [2]:
def mock_llm(state: MessagesState):
    return {"messages": [{"role": "ai", "content": "hello world"}]}

graph = StateGraph(MessagesState)
graph.add_node(mock_llm)
graph.add_edge(START, "mock_llm")
graph.add_edge("mock_llm", END)
graph = graph.compile()

graph.invoke({"messages": [{"role": "user", "content": "Hi~"}]})

{'messages': [HumanMessage(content='Hi~', additional_kwargs={}, response_metadata={}, id='21271a68-5dc8-4d61-913e-68a562d9cfe8'),
  AIMessage(content='hello world', additional_kwargs={}, response_metadata={}, id='b4889a49-34cf-43c1-9235-363153be5629', tool_calls=[], invalid_tool_calls=[])]}

## 测试qwen (langchain)

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
 
llm = ChatOpenAI(
    # 根据自己的需求配置，可以是环境变量，也可以是文本内容
    api_key="sk-3e43aba7e80343bc96fb5e7d549837ac", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus"
)
 
messages = [
    # 系统提示词：将英文翻译为中文
    SystemMessage("Translate the following from English into chinese"),
    HumanMessage("hi! I'm a PhD student"),
]
 
aimessages = llm.invoke(messages)
print(aimessages.content)

嗨！我是一名博士生


## langgraph + qwen

In [4]:
def qwen_llm(state: MessagesState):
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

graph = StateGraph(MessagesState)
graph.add_node("qwen", qwen_llm)
graph.add_edge(START, "qwen")
graph.add_edge("qwen", END)
graph = graph.compile()

graph.invoke({"messages": [{"role": "user", "content": "Hi~ Who are you?"}]})

{'messages': [HumanMessage(content='Hi~ Who are you?', additional_kwargs={}, response_metadata={}, id='25fd6349-dbbc-4b7c-a0be-f8b2c9d6a850'),
  AIMessage(content="Hello! 😊 I'm Qwen, a large-scale language model developed by Alibaba Cloud's Tongyi Lab. I can chat with you, help answer questions, write stories, create documents, code, and even express opinions. I love learning new things and helping out! What would you like to chat about? 🌟", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 14, 'total_tokens': 80, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-64f63262-bcc4-9835-9116-cab5261aa827', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bbb67-6e6d-7c32-b7bc-df9fc89e6fb1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 14, 'o

## complex langgraph structure

### 多节点，简单边（双 agent）

In [ ]:
# 算数 agent
def math_agent(state: MessagesState):
    system_prompt = SystemMessage("""
    你是一个普通的计算员，会做基本的数学运算。
    请根据用户的问题或者审核员的建议，直接给出答案。不要给出计算过程。
    """)
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)

    # 标记 response 名称
    response.name = "math_agent"
    return {"messages": [response]}


# 检查 agent
def check_agent(state: MessagesState):
    system_prompt = SystemMessage("""
    你是一个数学家，擅长检查各种数学问题。
    请检查 math_agent 的答案是否正确。
    如果正确，请给出 "correct"，并确保回答中没有"incorrect"这个词。
    如果错误，请给出新的计算建议，请给出 "incorrect"。
    """)
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)

    # 标记 response 名称
    response.name = "check_agent"
    return {"messages": [response]}



graph = StateGraph(MessagesState)
graph.add_node("math_agent", math_agent)
graph.add_node("check_agent", check_agent)
graph.add_edge(START, "math_agent")
graph.add_edge("math_agent", "check_agent")
graph.add_edge("check_agent", END)
graph = graph.compile()

# 生成图的流程
qa_async_ascii = graph.get_graph().print_ascii()
print(qa_async_ascii)


graph.invoke({"messages": [{"role": "user", "content": "1 + 1 = ?"}]})

 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
+------------+   
| math_agent |   
+------------+   
        *        
        *        
        *        
+-------------+  
| check_agent |  
+-------------+  
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    
None


{'messages': [HumanMessage(content='1 + 1 = ?', additional_kwargs={}, response_metadata={}, id='4c02d43e-3a7b-4df2-9342-64bd2cd4968a'),
  AIMessage(content='2', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 56, 'total_tokens': 57, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-92963ad5-7a51-9b4f-83c4-3a17b78e81c8', 'finish_reason': 'stop', 'logprobs': None}, name='math_agent', id='lc_run--019bbb88-3ddf-70a1-9815-dae2ecd49f27-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 56, 'output_tokens': 1, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  AIMessage(content='correct', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 78, 'total_to

In [6]:
graph.invoke({"messages": [{"role": "user", "content": "what is 3^{12345} (mod 100)?"}]})

{'messages': [HumanMessage(content='what is 3^{12345} (mod 100)?', additional_kwargs={}, response_metadata={}, id='ed16a73e-9415-4b07-a50c-a25324c0b160'),
  AIMessage(content='36', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 58, 'total_tokens': 60, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-707b890c-7506-962b-be5a-e8acd4efba89', 'finish_reason': 'stop', 'logprobs': None}, name='math_agent', id='lc_run--019bbb68-5337-7c13-9f97-42fa3ea91128-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 58, 'output_tokens': 2, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  AIMessage(content="We are asked to compute:\n\n$$\n3^{12345} \\mod 100\n$$\n\nand check whether the answer **36** is correct.\n\n---\n\

### 多节点，判断边（双 agent）

In [9]:
def should_continue(state: MessagesState):
    """
    根据审核结果，判断是否重新计算。
    返回下一个节点的名称。
    """
    last_message = state["messages"][-1]
    if 'incorrect' in last_message.content:
        print("审核不通过，打回中...")
        return "math_agent"
    else:
        print("审核通过，返回结果...")
        return "END"


In [15]:
workflow = StateGraph(MessagesState)

# 添加节点
workflow.add_node("math_agent", math_agent)
workflow.add_node("check_agent", check_agent)

# 添加边
workflow.add_edge(START, "math_agent")
workflow.add_edge("math_agent", "check_agent")
# 复杂判断
workflow.add_conditional_edges(
    "check_agent",
    should_continue,
    {
        "math_agent": "math_agent",
        "END": END,
    },
)
workflow = workflow.compile()

workflow.invoke({"messages": [{"role": "user", "content": "what is 3^{12345} (mod 100)?"}]})

审核不通过，打回中...
审核通过，返回结果...


{'messages': [HumanMessage(content='what is 3^{12345} (mod 100)?', additional_kwargs={}, response_metadata={}, id='ab597f2e-5a8a-4d68-ad19-aa66ad3e7ca1'),
  AIMessage(content='67', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 68, 'total_tokens': 70, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-394e8017-720e-9cb2-b8e7-137f229b0729', 'finish_reason': 'stop', 'logprobs': None}, name='math_agent', id='lc_run--019bbb88-62d2-73c3-9725-b495b0025f61-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 2, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}),
  AIMessage(content="incorrect\n\nTo compute $ 3^{12345} \\mod 100 $, we can use Euler's theorem and the Chinese Remainder Theorem.\n\nNo

In [17]:
# 生成图的流程图片
qa_async_ascii = workflow.get_graph().print_ascii()
print(qa_async_ascii)

 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
+------------+   
| math_agent |   
+------------+   
        *        
        *        
        *        
+-------------+  
| check_agent |  
+-------------+  
        .        
        .        
        .        
  +---------+    
  | __end__ |    
  +---------+    
None
